<a href="https://colab.research.google.com/github/chow-q/wav2Lip-webUI/blob/main/%E7%BD%91%E9%A1%B5%E7%89%88%E9%AB%98%E6%B8%85%E7%89%88ai%E4%BA%BA%E7%89%A9%E5%8F%A3%E5%9E%8B%E5%8C%B9%E9%85%8Dwav2lip_gfpgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/justinjohn0306/Wav2Lip.git
!git clone https://github.com/TencentARC/GFPGAN.git
!pip install -q gradio==3.50.2 yt_dlp ffmpeg-python librosa==0.8.0 batch_face
!python /content/GFPGAN/setup.py develop
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -P '/content/GFPGAN/experiments/pretrained_models'
%cd Wav2Lip
#download the pretrained model
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'

In [ ]:
# @title Default title text
!pip install basicsr
!pip install facexlib
!pip install -r ../GFPGANrequirements.txt
!pip install realesrgan

In [ ]:
import gradio as gr
import os
import cv2
from tqdm import tqdm
from os import path



def generate(video_in, audio_in):
    import cv2
    import numpy as np
    from tqdm import tqdm
    print(video_in)
    print(audio_in)
    outputPath="/content/Wav2Lip"
    !python inference.py --checkpoint_path checkpoints/wav2lip.pth --face '{video_in}' --audio '{audio_in}' --nosmooth --pads 0 15 0 0 --out_height=720
    #return f"/content/Wav2Lip/results/result_voice.mp4"
    print("把wav2lip生成的视频用fgpgan高清处理")
    #把wav2lip生成的视频用cv2把里面的每一帧画面都转成jpg保存起来
    inputVideoPath = "/content/Wav2Lip/results/result_voice.mp4"
    inputAudioPath = audio_in
    unProcessedFramesFolderPath = '/content/Wav2Lip/results/frames'
    if not os.path.exists(unProcessedFramesFolderPath):
      os.makedirs(unProcessedFramesFolderPath)
    vidcap = cv2.VideoCapture(inputVideoPath)
    numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    print("FPS: ", fps, "Frames: ", numberOfFrames)

    for frameNumber in tqdm(range(numberOfFrames)):
        _,image = vidcap.read()
        cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)
    print("用gfpgan高清修复每一帧画面")
    os.system(f"python /content/GFPGAN/inference_gfpgan.py -i {unProcessedFramesFolderPath} -o {outputPath} -v 1.4 -s 2 --bg_upsampler realesrgan")
    #restoredFrames
    restoredFramesPath = outputPath + '/restored_imgs/'
    processedVideoOutputPath = outputPath

    dir_list = os.listdir(restoredFramesPath)
    dir_list.sort()



    batch = 0
    batchSize = 300

    for i in tqdm(range(0, len(dir_list), batchSize)):
      img_array = []
      start, end = i, i+batchSize
      print("processing ", start, end)
      for filename in  tqdm(dir_list[start:end]):
          filename = restoredFramesPath+filename;
          img = cv2.imread(filename)
          if img is None:
            continue
          height, width, layers = img.shape
          size = (width,height)
          img_array.append(img)


      out = cv2.VideoWriter(processedVideoOutputPath+'/batch_'+str(batch).zfill(4)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
      batch = batch + 1

      for i in range(len(img_array)):
        out.write(img_array[i])
      out.release()
    #合并视频和音频
    concatTextFilePath = outputPath + "/concat.txt"
    concatTextFile=open(concatTextFilePath,"w")
    for ips in range(batch):
      concatTextFile.write("file batch_" + str(ips).zfill(4) + ".avi\n")
    concatTextFile.close()

    concatedVideoOutputPath = outputPath + "/concated_output.avi"
    print(concatTextFilePath,"2:",concatedVideoOutputPath,"3:",)
    os.system(f"ffmpeg -y -f concat -i {concatTextFilePath} -c copy {concatedVideoOutputPath}")

    finalProcessedOuputVideo = processedVideoOutputPath+'/final_with_audio.avi'
    print(concatTextFilePath,"4:",concatedVideoOutputPath,"5:",finalProcessedOuputVideo)
    os.system(f"ffmpeg -y -i {concatedVideoOutputPath} -i {audio_in} -map 0 -map 1:a -c:v copy -shortest {finalProcessedOuputVideo}")


    return f"/content/Wav2Lip/final_with_audio.avi"


with gr.Blocks() as demo:
  with gr.Row():
    with gr.Column():
      input_video = gr.Video(label="Input Video")
      input_audio = gr.Audio(label="Input Audio", type='filepath')
      input_generate_button = gr.Button(label="Generate")
    with gr.Column():
        video_out = gr.Video(label="Output Video")
    input_generate_button.click(generate, inputs=[input_video, input_audio], outputs=[video_out])

demo.queue().launch(inline=False, share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://255f69f444f534052f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
/tmp/gradio/14a65644e9fee50126da2375a50e84f138938d28/kimk_7s_raw.mp4
/tmp/gradio/3406fbe010083d7b8ca2aa5ab385c7c1d9528929/input_audio-0-100.wav
Using cuda for inference.
Load checkpoint from: checkpoints/wav2lip.pth
Models loaded
Reading video frames...
Number of frames available for inference: 495
(80, 966)
Length of mel chunks: 358
  0% 0/3 [00:00<?, ?it/s]face detect time: 49.35104751586914
100% 3/3 [01:08<00:00, 22.69s/it]
wav2lip prediction time: 68.06535959243774
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/us

100%|██████████| 358/358 [00:04<00:00, 79.87it/s]


用gfpgan高清修复每一帧画面


  0%|          | 0/2 [00:00<?, ?it/s]

processing  0 300


...


 50%|█████     | 1/2 [00:13<00:13, 13.96s/it]

processing  300 600



100%|██████████| 2/2 [00:16<00:00,  8.18s/it]


/content/Wav2Lip/concat.txt 2: /content/Wav2Lip/concated_output.avi 3:
/content/Wav2Lip/concat.txt 4: /content/Wav2Lip/concated_output.avi 5: /content/Wav2Lip/final_with_audio.avi


/usr/local/lib/python3.10/dist-packages/gradio/components/video.py:329: UserWarning: Video does not have browser-compatible container or codec. Converting to mp4
  warnings.warn(
